In [1]:
import gpn.model
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import torch
from transformers import AutoModel, AutoModelForMaskedLM, AutoTokenizer
import h5py
import sys
from tqdm import tqdm
import numpy as np
sys.path.append('../')
import utils
import os
from tqdm import tqdm
import glob
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
 
model = AutoModel.from_pretrained("/home/ztang/multitask_RNA/model/GPN_human/checkpoint-2000000").to('cuda')
tokenizer = AutoTokenizer.from_pretrained("/home/ztang/multitask_RNA/model/GPN_human/checkpoint-2000000")
model.eval();

file_list = glob.glob('/home/ztang/multitask_RNA/data/eclip/*.h5')


/home/ztang/.conda/envs/gpn_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at /home/ztang/multitask_RNA/model/GPN_human/checkpoint-2000000 were not used when initializing ConvNetModel: ['cls.decoder.0.bias', 'cls.decoder.2.weight', 'cls.decoder.3.weight', 'cls.decoder.2.bias', 'cls.decoder.3.bias', 'cls.decoder.0.weight']
- This IS expected if you are initializing ConvNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ConvNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificatio

In [2]:
for file in file_list:
    tf_name = file.split('/')[-1][:-7]
    gpn_output = h5py.File('/home/ztang/multitask_RNA/data/eclip/GPN/'+tf_name+'_200.h5','w')
    batch_size = 32
    file = h5py.File(file,'r')
    for label in ('train','valid','test'):
        output_cache = []  
        for i in tqdm(range(0,len(file['X_'+label]),batch_size)):
            seq = file['X_'+label][i:i+batch_size].astype('int')
            seq = np.transpose(seq,(0,2,1))
            seq = utils.onehot_to_seq(seq)
            input_ids = tokenizer(seq, return_tensors="pt", return_attention_mask=False, return_token_type_ids=False)["input_ids"]
            with torch.no_grad():
                output_seq = model(input_ids.to('cuda')).last_hidden_state.cpu().detach().numpy()
            output_cache.extend(output_seq)
        gpn_output.create_dataset(name='X_'+label,data = np.array(output_cache),dtype = 'float32')
        gpn_output.create_dataset(name='Y_'+label,data = file['Y_'+label][:],dtype='int') 
    gpn_output.close()

  0%|          | 0/715 [00:00<?, ?it/s]/home/ztang/.conda/envs/gpn_env/lib/python3.9/site-packages/torch/nn/modules/conv.py:309: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv1d(input, weight, bias, self.stride,
100%|██████████| 91/91 [00:03<00:00, 29.18it/s]
